# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [82]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [83]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra
['/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-15-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-22-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-09-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-18-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-04-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-01-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-27-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-modeling-with-Apache-Cassandra/event_data/2018-11-10-events.csv', '/Users/lauracalcagni/Documents/GitHub/Data-m

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [84]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [85]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [86]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [87]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [88]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [95]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4


In [89]:
query = """CREATE TABLE IF NOT EXISTS song_by_session_item (
    sessionId int,
    itemInSession int,
    song text,
    length float,
    artist text,
    PRIMARY KEY (sessionId, itemInSession)
)"""

try:
    session.execute(query)
except Exception as e:
    print(e)

In [90]:
# Populates the table with data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO song_by_session_item (sessionId, itemInSession, song, length, artist)
        VALUES (%s, %s, %s, %s, %s)
        """
        
        sessionId = int(line[8])
        itemInSession = int(line[3])
        song = line[9]
        length = float(line[5])
        artist = line[0]

        session.execute(query, (sessionId, itemInSession, song, length, artist))

In [91]:
query = "SELECT artist, song, length FROM song_by_session_item WHERE sessionId=338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


In [94]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

In [99]:
query = """
    CREATE TABLE IF NOT EXISTS song_by_user_session (
        userId int,
        firstName text,
        lastName text,
        sessionId int,
        itemInSession int,
        song text,
        artist text,
        PRIMARY KEY ((userId, sessionId), itemInSession))
    WITH CLUSTERING ORDER BY (itemInSession ASC)
    """

try:
    session.execute(query)
except Exception as e:
    print(e)



In [93]:
# Populates the table with data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO song_by_user_session (userId, firstName, lastName, sessionId, itemInSession, song, artist)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        
        userId = int(line[10])
        firstName = line[1]
        lastName = line[4]
        sessionId = int(line[8])
        itemInSession = int(line[3])
        song = line[9]
        artist = line[0]

        session.execute(query, (userId, firstName, lastName, sessionId, itemInSession, song, artist))

InvalidRequest: Error from server: code=2200 [Invalid query] message="table song_by_user_session does not exist"

In [ ]:
query = "SELECT artist, song, firstName, lastName FROM song_by_user_session WHERE sessionId=182 and userId=10"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'


In [ ]:
query = """
    CREATE TABLE IF NOT EXISTS users_by_song (
        song text,
        userId int,
        firstName text,
        lastName text,
        PRIMARY KEY (song, userId)
    )"""


try:
    session.execute(query)
except Exception as e:
    print(e)

In [ ]:
# Populates the table with data from the CSV file
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = """
        INSERT INTO users_by_song (userId, firstName, lastName, song)
        VALUES (%s, %s, %s, %s)
        """
        
        userId = int(line[10])
        firstName = line[1]
        lastName = line[4]
        song = line[9]

        session.execute(query, (userId, firstName, lastName, song))

In [ ]:
query = "SELECT firstName, lastName FROM users_by_song WHERE song = 'All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Drop the tables before closing out the sessions

In [ ]:
## Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS sparkify.song_by_session_item")
session.execute("DROP TABLE IF EXISTS sparkify.song_by_user_session")
session.execute("DROP TABLE IF EXISTS sparkify.users_by_song")

### Close the session and cluster connection¶

In [ ]:
session.shutdown()
cluster.shutdown()